In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ESN
import pandas as pd

In [ ]:
def set_seed(seed=42):
    """Making the seed (for random values) variable if None"""

    # Set the seed
    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
    try:
        np.random.seed(seed)
    except Exception as e:
        print( "!!! WARNING !!!: Seed was not set correctly.")
        print( "!!! Seed that we tried to use: "+str(seed))
        print( "!!! Error message: "+str(e))
        seed = None
    print( "Seed used for random values:", seed)
    return seed

In [ ]:
df =  pd.read_excel(r'C:\Users\INFO-DSK-02\Desktop\Lorentz Multi Dimension Prediction-Phase-2\Final_Version\3D_ReservoirComputing\Input\Lorentz data testing and training.xlsx', index = False)

In [ ]:
initLen = 50 
trainLen = initLen + 1950
testLen = 800

In [ ]:
data_in = df[['x','y','z']]
data_T  =df['t']

In [ ]:
data_in = np.array(data_in)
data_t = np.array(data_T)

In [ ]:
train_in = np.array(data_in[0:trainLen])
train_out = np.array(data_in[0+10:trainLen+10])
valid_in = np.array(data_in[trainLen:trainLen+testLen])
valid_out = np.array(data_in[trainLen+10:trainLen+testLen+10])

In [ ]:
n_r = [300,400,500,800] # number of recurrent units
l_r =[ 0.3,0.2,0.1,0.4] # leaking rate (=1/time_constant_of_neurons)
s_r = [1,1.2,1.4,1.5,1.6]
r_c = [1e-3,1e-2,1e-1]
train_in = train_in
train_out = train_out
valid_in = valid_in
valid_out = valid_out
input_bias = True
dim_inp = 3
n_outputs = 3
initLen = initLen

In [ ]:
def ESN_TUNE(n_reservoir, leak_rate, regularization_coef, spectral_radius, n_inputs,n_outputs, initLen, train_in, train_out, valid_in, valid_out):
    Best_Parameters = []
    input_bias = True
    nr = n_reservoir
    l_r = leak_rate
    r_c = regularization_coef
    s_r = spectral_radius
    dim_inp = n_inputs
    n_outputs = n_outputs
    dim_inp = n_inputs
    initLen = initLen
    for N in n_r:
        for L in l_r:
            for R in r_c:
                for i in range(len(s_r)):
                    parameters = []
                    N =  N#100
                    spectral_radius = s_r[i]
                    input_scaling = 1. # Scaling of input matrix
                    proba_non_zero_connec_W = 0.2 # Sparsity of recurrent matrix: Perceptage of non-zero connections in W matrix
                    proba_non_zero_connec_Win = 1. # Sparsity of input matrix
                    proba_non_zero_connec_Wfb = 1.
                        ### Generating random weight matrices with custom method
                    W = np.random.rand(N,N) - 0.5
                    if input_bias:
                        Win = np.random.rand(N,dim_inp+1) - 0.5
                    else:
                        Win = np.random.rand(N,dim_inp) - 0.5
                    Wfb = np.random.rand(N,n_outputs) - 0.5
                    mask = np.random.rand(N,N) # create a mask Uniform[0;1]
                    W[mask > proba_non_zero_connec_W] = 0 # set to zero some connections given by the mask
                    mask = np.random.rand(N,Win.shape[1])
                    Win[mask > proba_non_zero_connec_Win] = 0
                    Win = Win * input_scaling
                    original_spectral_radius = np.max(np.abs(np.linalg.eigvals(W)))
                    W = W * (spectral_radius / original_spectral_radius)
                    reservoir = ESN.ESN(lr=L, W=W, Win=Win, input_bias=input_bias, ridge=R, Wfb=None, fbfunc=None)
                    internal_trained = reservoir.train(inputs=[train_in], teachers=[train_out], wash_nr_time_step=initLen, verbose=False)
                    output_pred, internal_pred = reservoir.run(inputs=[valid_in,], reset_state=False)
                    df_pred = pd.DataFrame(output_pred[0])
                    df_test_out = pd.DataFrame(valid_out)
                    X_MSE = np.mean((df_test_out[0][:] - df_pred[0])**2)
                    Y_MSE = np.mean((df_test_out[1][:] - df_pred[1])**2)
                    Z_MSE = np.mean((df_test_out[2][:] - df_pred[2])**2)
#                     print('For ', N,L,R,s_r[i], 'parameters:: ', 'X, Y, Z MSE = ', X_MSE, Y_MSE, Z_MSE)
#                     print('RMSE = ', np.sqrt(X_MSE**2+Y_MSE**2+Z_MSE**2))
                    RMSE = np.sqrt(X_MSE**2+Y_MSE**2+Z_MSE**2)
                    parameters.append({'n_reservoir':N,'leaky_rate':L,'regularization_coef':R,'spectral_radius' :s_r[i]})
                    Best_Parameters.append([parameters, RMSE, X_MSE])
    df = pd.DataFrame(Best_Parameters)
    return df.iloc[np.argmin(np.array(df[1]))][0][0]

In [ ]:
import time
t1 = time.perf_counter()
best_parameters = ESN_TUNE(n_reservoir= [300,400,500],
         leak_rate = [ 0.3,0.2,0.1], 
         regularization_coef = [1e-3,1e-2,1e-1],
         spectral_radius = [1,1.2,1.4,1.5], 
         n_inputs =3,
         n_outputs = 3, 
         initLen  = initLen, 
         train_in = train_in, 
         train_out = train_out, 
         valid_in = valid_in, 
         valid_out = valid_out)
t2 = time.perf_counter()
print(t2-t1)

In [ ]:
print('Best Parameters :', best_parameters)